# Join RSEI score reports to TRI data

The EPA's Risk-Screening Environmental Indicators (RSEI) model incorporates info from the TRI on the amount of toxic chemical released, with factors such as the chemical's fate and transport through the environment, each chemical's relative toxicity, and potential human exposure.

The [Easy RSEI Dashboard](https://edap.epa.gov/public/extensions/EasyRSEI/EasyRSEI.html) lets users make various selections and generate reports for RSEI scores and modeled hazards. These numbers are a better way to visualize TRI facilities than just release totals, as there are many factors that go into determining each facility's risk factor.

I selected all TRI reporting facilties in 2018, and downloaded excel workbooks for the five RSEI value options. Each record represents one facility and reports its RSEI value, it also links to that facility's RSEI report which further details the score. There is a TRIFID column that I can use to join to my existing TRI geojson.

In [15]:
# import packages
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import mapping
from functools import reduce

import warnings
warnings.simplefilter(action='ignore')

In [16]:
# read tri gdf in
tri = gpd.read_file('../data/tri-facilities.geojson')
tri.head()

,TRIFID,YEAR,FACILITY_NAME,STREET_ADDRESS,CITY,COUNTY,ST,ZIP,BIA,TRIBE,...,TOTAL_CAR_OFF_SITE_RELEASE,TOTAL_CAR_RELEASE,TOTAL_TRI_WATER_RELEASE,TOTAL_TRI_AIR_RELEASE,TOTAL_TRI_LAND_RELEASE,TOTAL_TRI_LAND_RELEASE_FILTER,TOTAL_TRI_ON_SITE_RELEASE,TOTAL_TRI_OFF_SITE_RELEASE,TOTAL_TRI_RELEASE,geometry
0,00608DCRBNRD3KM,2018,IDI CARIBE INC,PR3 KM 151.8 AGUIRRE,SALINAS,SALINAS,PR,751,,,...,0.0000,13143.0000,0.0,13143.38,0.0,0.0,13143.38,7.15,13150.53,POINT (-66.23194 17.97278)
1,0060WHPNTRCARR1,2018,HP INTERNATIONAL TRADING BV (PUERTO RICO BRANC...,"CARR 110, KM. 5.1",AGUADILLA,AGUADILLA,PR,603,,,...,NaN,NaN,0.0,181.40,0.0,0.0,181.40,0.00,181.40,POINT (-67.13655 18.45647)
2,00610BXTRHROAD4,2018,EDWARDS LIFESCIENCES TECHNOLOGY SARL,STATE RD 402 KM 1.4 N LAS MARIAS IND US TRIAL ...,ANASCO,ANASCO,PR,610,,,...,0.0000,580.0000,0.0,580.00,0.0,0.0,580.00,0.00,580.00,POINT (-67.13682 18.29298)
3,00610CRBGNCARR4,2018,GE INTERNATIONAL OF PR LLC,STATE RD 402 KM 1.5 INDUSTRIAL ZONE,ANASCO,ANASCO,PR,610,,,...,1.0000,1.2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-67.14019 18.29244)
4,00612PRPCMPR681,2018,PREPA-CAMBALACHE COMBUSTION TURBINE PLANT,PR-681 KM 0.5,ARECIBO,ARECIBO,PR,612,,,...,1.8612,44.0612,0.0,0.00,0.0,0.0,0.00,0.00,0.00,POINT (-66.69940 18.47110)


In [31]:
# read RSEI score tables in
scores = pd.read_excel('../data/rsei-tables/2018-rsei-scores.xlsx')
cancer_scores = pd.read_excel('../data/rsei-tables/2018-rsei-cancer-scores.xlsx')
non_cancer_scores = pd.read_excel('../data/rsei-tables/2018-rsei-non-cancer-scores.xlsx')
modeled_hazard = pd.read_excel('../data/rsei-tables/2018-rsei-modeled-hazard.xlsx')
modeled_pounds = pd.read_excel('../data/rsei-tables/2018-rsei-modeled-pounds.xlsx')

print(scores.columns)
print(cancer_scores.columns)
print(non_cancer_scores.columns)
print(modeled_hazard.columns)
print(modeled_pounds.columns)

Index(['RSEI Score', 'TRIFID', 'Name', 'Street', 'City', 'State/Territory',
       'RSEI Facility Report'],
      dtype='object')
Index(['RSEI Score-Cancer', 'TRIFID', 'Name', 'Street', 'City',
       'State/Territory', 'RSEI Facility Report'],
      dtype='object')
Index(['RSEI Score-Noncancer', 'TRIFID', 'Name', 'Street', 'City',
       'State/Territory', 'RSEI Facility Report'],
      dtype='object')
Index(['RSEI Modeled Hazard', 'TRIFID', 'Name', 'Street', 'City',
       'State/Territory', 'RSEI Facility Report'],
      dtype='object')
Index(['RSEI Modeled Pounds', 'TRIFID', 'Name', 'Street', 'City',
       'State/Territory', 'RSEI Facility Report'],
      dtype='object')


I need to join the RSEI data to my TRI geodataframe, with a column for each of the RSEI values, as well as the RSEI facility report. I can join based on the TRIFID.

In [32]:
# first I will reduce each rsei dataframe to just its RSEI value, TRIFID, and RSEI facility report
scores = scores.filter(['RSEI Score', 'TRIFID', 'RSEI Facility Report'])
cancer_scores = cancer_scores.filter(['RSEI Score-Cancer', 'TRIFID', 'RSEI Facility Report'])
non_cancer_scores = non_cancer_scores.filter(['RSEI Score-Noncancer', 'TRIFID', 'RSEI Facility Report'])
modeled_hazard = modeled_hazard.filter(['RSEI Modeled Hazard', 'TRIFID', 'RSEI Facility Report'])
modeled_pounds = modeled_pounds.filter(['RSEI Modeled Pounds', 'TRIFID', 'RSEI Facility Report'])

In [33]:
# create list of tri and all rsei dataframes to be merged
data_frames = [tri, scores, cancer_scores, non_cancer_scores, modeled_hazard, modeled_pounds]

# join list of dataframes
tri_joined = reduce(lambda left,right: pd.merge(left,right,on=['TRIFID'],
                                            how='outer'), data_frames)

# inspect results
tri_joined.head()

,TRIFID,YEAR,FACILITY_NAME,STREET_ADDRESS,CITY,COUNTY,ST,ZIP,BIA,TRIBE,...,RSEI Score,RSEI Facility Report_x,RSEI Score-Cancer,RSEI Facility Report_y,RSEI Score-Noncancer,RSEI Facility Report_x,RSEI Modeled Hazard,RSEI Facility Report_y,RSEI Modeled Pounds,RSEI Facility Report
0,00608DCRBNRD3KM,2018,IDI CARIBE INC,PR3 KM 151.8 AGUIRRE,SALINAS,SALINAS,PR,751,,,...,2.037185,https://enviro.epa.gov/enviro/rsei.html?facid=...,0.0000,https://enviro.epa.gov/enviro/rsei.html?facid=...,2.037185,https://enviro.epa.gov/enviro/rsei.html?facid=...,4.603850e+04,https://enviro.epa.gov/enviro/rsei.html?facid=...,13143.380000,https://enviro.epa.gov/enviro/rsei.html?facid=...
1,0060WHPNTRCARR1,2018,HP INTERNATIONAL TRADING BV (PUERTO RICO BRANC...,"CARR 110, KM. 5.1",AGUADILLA,AGUADILLA,PR,603,,,...,1.761933,https://enviro.epa.gov/enviro/rsei.html?facid=...,0.0000,https://enviro.epa.gov/enviro/rsei.html?facid=...,1.761933,https://enviro.epa.gov/enviro/rsei.html?facid=...,3.265200e+04,https://enviro.epa.gov/enviro/rsei.html?facid=...,181.400000,https://enviro.epa.gov/enviro/rsei.html?facid=...
2,00610BXTRHROAD4,2018,EDWARDS LIFESCIENCES TECHNOLOGY SARL,STATE RD 402 KM 1.4 N LAS MARIAS IND US TRIAL ...,ANASCO,ANASCO,PR,610,,,...,901403.700000,https://enviro.epa.gov/enviro/rsei.html?facid=...,901403.7000,https://enviro.epa.gov/enviro/rsei.html?facid=...,9.833503,https://enviro.epa.gov/enviro/rsei.html?facid=...,6.380000e+09,https://enviro.epa.gov/enviro/rsei.html?facid=...,580.000000,https://enviro.epa.gov/enviro/rsei.html?facid=...
3,00610CRBGNCARR4,2018,GE INTERNATIONAL OF PR LLC,STATE RD 402 KM 1.5 INDUSTRIAL ZONE,ANASCO,ANASCO,PR,610,,,...,0.434199,https://enviro.epa.gov/enviro/rsei.html?facid=...,0.0000,https://enviro.epa.gov/enviro/rsei.html?facid=...,0.434199,https://enviro.epa.gov/enviro/rsei.html?facid=...,4.600000e+03,https://enviro.epa.gov/enviro/rsei.html?facid=...,0.200000,https://enviro.epa.gov/enviro/rsei.html?facid=...
4,00612PRPCMPR681,2018,PREPA-CAMBALACHE COMBUSTION TURBINE PLANT,PR-681 KM 0.5,ARECIBO,ARECIBO,PR,612,,,...,55.560600,https://enviro.epa.gov/enviro/rsei.html?facid=...,55.5606,https://enviro.epa.gov/enviro/rsei.html?facid=...,0.000000,https://enviro.epa.gov/enviro/rsei.html?facid=...,1.679640e+07,https://enviro.epa.gov/enviro/rsei.html?facid=...,44.079998,https://enviro.epa.gov/enviro/rsei.html?facid=...


Now I have multiple columns for the RSEI facility report. I need to drop any duplicates.

In [34]:
# get list of column names that contain RSEI Facility Report
# duplicates were renamed during the merge
column_names = tri_joined.columns.str.contains('RSEI Facility Report', case=False)
report_columns = tri_joined.iloc[:, column_names]
list(report_columns.columns)

['RSEI Facility Report_x',
 'RSEI Facility Report_y',
 'RSEI Facility Report_x',
 'RSEI Facility Report_y',
 'RSEI Facility Report']

In [35]:
tri_joined = tri_joined.drop(['RSEI Facility Report_x', 'RSEI Facility Report_y'], axis=1)
list(tri_joined.columns)

['TRIFID',
 'YEAR',
 'FACILITY_NAME',
 'STREET_ADDRESS',
 'CITY',
 'COUNTY',
 'ST',
 'ZIP',
 'BIA',
 'TRIBE',
 'LATITUDE',
 'LONGITUDE',
 'INDUSTRY_SECTOR',
 'CHEMICAL',
 'PARENT_CO_NAME',
 'ON_SITE_RELEASE_TOTAL',
 'OFF_SITE_RELEASE_TOTAL',
 'TOTAL_WATER_RELEASE',
 'TOTAL_AIR_RELEASE',
 'TOTAL_LAND_RELEASE',
 'TOTAL_LAND_RELEASE_FILTER',
 'TOTAL_DIOXIN_WATER_RELEASE',
 'TOTAL_DIOXIN_AIR_RELEASE',
 'TOTAL_DIOXIN_LAND_RELEASE',
 'TOTAL_DIOXIN_LAND_RELEASE_FILTER',
 'TOTAL_DIOXIN_ON_SITE_RELEASE',
 'TOTAL_DIOXIN_OFF_SITE_RELEASE',
 'TOTAL_DIOXIN_RELEASE',
 'TOTAL_CAR_WATER_RELEASE',
 'TOTAL_CAR_AIR_RELEASE',
 'TOTAL_CAR_LAND_RELEASE',
 'TOTAL_CAR_LAND_RELEASE_FILTER',
 'TOTAL_CAR_ON_SITE_RELEASE',
 'TOTAL_CAR_OFF_SITE_RELEASE',
 'TOTAL_CAR_RELEASE',
 'TOTAL_TRI_WATER_RELEASE',
 'TOTAL_TRI_AIR_RELEASE',
 'TOTAL_TRI_LAND_RELEASE',
 'TOTAL_TRI_LAND_RELEASE_FILTER',
 'TOTAL_TRI_ON_SITE_RELEASE',
 'TOTAL_TRI_OFF_SITE_RELEASE',
 'TOTAL_TRI_RELEASE',
 'geometry',
 'RSEI Score',
 'RSEI Score-Can

In [36]:
# replace spaces in column names with underscores
tri_joined.columns = tri_joined.columns.str.replace(' ', '_')

# replace dashes in column names with underscores
tri_joined.columns = tri_joined.columns.str.replace('-', '_')

In [37]:
# export file to geojson
tri_joined.to_file("../data/TRI-facilities-rsei.geojson", driver='GeoJSON')